<a href="https://colab.research.google.com/github/tsabbir96/spam-article-checker/blob/main/Malicious_URL_Detector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#import the necessary libraries
from functools import wraps
from flask import Flask
from flask import request, Response
from subprocess import call
from flask import render_template
import pickle

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import random
from sklearn.feature_extraction.text import TfidfVectorizer
import sys
import os
from sklearn import model_selection, naive_bayes, svm
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import math
from collections import Counter

app = Flask(__name__)

from google.colab import drive
drive.mount("/content/gdrive")


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [ ]:
#create tokenizer
def getTokens(input):
	tokensBySlash = str(input.encode('utf-8')).split('/')	#get tokens after splitting by slash
	allTokens = []
	for i in tokensBySlash:
		tokens = str(i).split('-')	#get tokens after splitting by dash
		tokensByDot = []
		for j in range(0,len(tokens)):
			tempTokens = str(tokens[j]).split('.')	#get tokens after splitting by dot
			tokensByDot = tokensByDot + tempTokens
		allTokens = allTokens + tokens + tokensByDot
	allTokens = list(set(allTokens))	#remove redundant tokens
	if 'com' in allTokens:
		allTokens.remove('com')	#removing .com since it occurs a lot of times and it should not be included in our features
	return allTokens


	

In [ ]:
allurlsdata = pd.read_csv('./gdrive/My Drive/urldata.csv')
allurlsdata



,Unnamed: 0,url,label,result
0,0,https://www.google.com,benign,0
1,1,https://www.youtube.com,benign,0
2,2,https://www.facebook.com,benign,0
3,3,https://www.baidu.com,benign,0
4,4,https://www.wikipedia.org,benign,0
...,...,...,...,...
450171,450171,http://ecct-it.com/docmmmnn/aptgd/index.php,malicious,1
450172,450172,http://faboleena.com/js/infortis/jquery/plugin...,malicious,1
450173,450173,http://faboleena.com/js/infortis/jquery/plugin...,malicious,1
450174,450174,http://atualizapj.com/,malicious,1


In [ ]:
## Checking for Missing values
allurlsdata.isnull().sum()


Unnamed: 0    0
url           0
label         0
result        0
dtype: int64

In [ ]:
# dropping ALL duplicte values 
allurlsdata.drop_duplicates(subset ="url", 
                     keep = False, inplace = True)
allurlsdata.shape

(450176, 4)

In [ ]:
# Change all the text to lower case.
allurlsdata['url'] = [entry.lower() for entry in allurlsdata['url']]

In [ ]:
m = allurlsdata[allurlsdata["label"] == "malicious" ]
m5k = (m.head(5000)).reset_index() #take first 5000 rows and create a dataframe
m5k 

,index,Unnamed: 0,url,label,result
0,345738,345738,http://atualizacaodedados.online,malicious,1
1,345739,345739,http://webmasteradmin.ukit.me/,malicious,1
2,345740,345740,http://stcdxmt.bigperl.in/klxtv/apps/uk/,malicious,1
3,345741,345741,https://tubuh-syarikat.com/plugins/fields/files/,malicious,1
4,345742,345742,http://rolyborgesmd.com/exceword/excel.php?.ra...,malicious,1
...,...,...,...,...,...
4995,350733,350733,http://wwwsn.cibadetran.portaldocooperado.com,malicious,1
4996,350734,350734,http://www.pousadavieiras.com.br/sqqq/boa/,malicious,1
4997,350735,350735,http://catherinetruskolawski.com/bright/newart...,malicious,1
4998,350736,350736,http://www.kasp.pp.ua/corex/access.htm,malicious,1


In [ ]:
b = allurlsdata[allurlsdata["label"] == "benign" ]
b5k = (b.head(5000)).reset_index() #take first 5000 rows and create a dataframe
b5k

,index,Unnamed: 0,url,label,result
0,0,0,https://www.google.com,benign,0
1,1,1,https://www.youtube.com,benign,0
2,2,2,https://www.facebook.com,benign,0
3,3,3,https://www.baidu.com,benign,0
4,4,4,https://www.wikipedia.org,benign,0
...,...,...,...,...,...
4995,4995,4995,https://www.athletics.concordia.ca/,benign,0
4996,4996,4996,https://www.athletics.concordia.ca/intercolleg...,benign,0
4997,4997,4997,https://www.athletics.concordia.ca/intercolleg...,benign,0
4998,4998,4998,https://www.athletics.concordia.ca/intercolleg...,benign,0


In [ ]:
allurlsdata = pd.concat([b5k,m5k])#concatenate both the dataset to create one single dataset
allurlsdata = allurlsdata.sample(frac=1).reset_index(drop=True)
allurlsdata = allurlsdata.sample(frac=1).reset_index(drop=True)

filename = 'allurlsdata.sav'
pickle.dump(allurlsdata, open(filename, 'wb'))

from google.colab import files
files.download('allurlsdata.sav')

allurlsdata



,index,Unnamed: 0,url,label,result
0,4801,4801,https://www.askye.wordpress.com/2007/08/05/fam...,benign,0
1,2769,2769,https://www.archiver.rootsweb.ancestry.com/th/...,benign,0
2,2817,2817,https://www.archiver.rootsweb.ancestry.com/th/...,benign,0
3,346113,346113,https://fb-secure-2018.000webhostapp.com/recov...,malicious,1
4,347048,347048,http://d14bullion.com/og/sky/index.php,malicious,1
...,...,...,...,...,...
9995,4819,4819,https://www.aspecialthing.com/forum/f27/check-...,benign,0
9996,348355,348355,http://www.salintoshourt.com/wp-boa/e31003deee...,malicious,1
9997,2290,2290,https://www.focus.cn,benign,0
9998,4380,4380,https://www.artists.letssingit.com/gwen-stefan...,benign,0


In [ ]:
#train the tokenizer
Tfidf_vect = TfidfVectorizer(tokenizer=getTokens)
#fit the tokenizer
Tfidf_vect.fit(allurlsdata['url'])

#transform the url data into vectors
X = Tfidf_vect.transform(allurlsdata['url'])
print(X)

#save file
filename = 'tokenizer.sav'
pickle.dump(Tfidf_vect, open(filename, 'wb'))

from google.colab import files
files.download('tokenizer.sav')

/usr/local/lib/python3.6/dist-packages/sklearn/feature_extraction/text.py:507: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"


  (0, 23284)	0.7046453070532828
  (0, 21342)	0.11017088179713781
  (0, 10231)	0.6533257881194882
  (0, 6826)	0.21333033894924944
  (0, 4830)	0.11623795485498711
  (0, 0)	0.07403841546659279
  (1, 25157)	0.4183940509074695
  (1, 21342)	0.07395221089189398
  (1, 20692)	0.4183940509074695
  (1, 11401)	0.25711349906546477
  (1, 9993)	0.4183940509074695
  (1, 9992)	0.3982431280795975
  (1, 4829)	0.09112973446536853
  (1, 2212)	0.47299320384074073
  (1, 26)	0.10083781557443759
  (1, 0)	0.04969829074046116
  (2, 23840)	0.6813048962187198
  (2, 21342)	0.10652162220878604
  (2, 12982)	0.6813048962187198
  (2, 6826)	0.20626406361226557
  (2, 4830)	0.11238773178001867
  (2, 0)	0.07158599434460032
  (3, 25130)	0.4906234163666401
  (3, 21342)	0.0767086843130707
  (3, 20542)	0.4906234163666401
  :	:
  (9996, 0)	0.06818984409747006
  (9997, 19049)	0.3646613086667649
  (9997, 19039)	0.2694504863165899
  (9997, 18610)	0.3295530304137267
  (9997, 16881)	0.26638359765846575
  (9997, 16330)	0.364661308666

In [ ]:
#this section will turn the labels into 0 and 1
#0 for benign
#1 for malicious
from sklearn.preprocessing import LabelEncoder
Encoder = LabelEncoder()
y = Encoder.fit_transform(allurlsdata['label'])
print(y)
print(allurlsdata["label"])

[0 1 0 ... 1 0 0]
0          benign
1       malicious
2          benign
3          benign
4          benign
          ...    
9995    malicious
9996       benign
9997    malicious
9998       benign
9999       benign
Name: label, Length: 10000, dtype: object


In [ ]:
#split into train test data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42)

In [ ]:
## training - LogisticRegression
logisticRegr = LogisticRegression()
logisticRegr.fit(X_train,y_train)

# predict the labels on validation dataset
predictions_LR = logisticRegr.predict(X_test)
# Use accuracy_score function to get the accuracy
print("LR Accuracy Score -> ",accuracy_score(predictions_LR, y_test)*100)

LR Accuracy Score ->  99.32


In [ ]:
# fit the training dataset on the NB classifier
Naive = naive_bayes.MultinomialNB()
Naive.fit(X_train,y_train)
# predict the labels on validation dataset
predictions_NB = Naive.predict(X_test)
# Use accuracy_score function to get the accuracy
print("Naive Bayes Accuracy Score -> ",accuracy_score(predictions_NB, y_test)*100)

Naive Bayes Accuracy Score ->  98.9


In [ ]:
## training - RandomForestClassifier
rfc = RandomForestClassifier()
rfc.fit(X_train,y_train)

# predict the labels on validation dataset
predictions_RFC = rfc.predict(X_test)
# Use accuracy_score function to get the accuracy
print("RFC Accuracy Score -> ",accuracy_score(predictions_RFC, y_test)*100)

RFC Accuracy Score ->  99.56


In [ ]:
#Save all the models
filename = 'naive_url.sav'
pickle.dump(Naive, open(filename, 'wb'))
filename = 'RFC_url.sav'
pickle.dump(rfc, open(filename, 'wb'))
filename = 'LogReg_url.sav'
pickle.dump(logisticRegr, open(filename, 'wb'))

In [ ]:
from google.colab import files
files.download('naive_url.sav')
files.download('LogReg_url.sav')
files.download('RFC_url.sav')